### Imports

In [1]:
import math, random
import numpy as np
import matplotlib.pyplot as plt
import sys
import statistics
from IPython.display import Markdown, clear_output
import ipywidgets as widgets
%matplotlib inline
%matplotlib widget
plt.rcParams['font.size'] = 14

### Problem setup

The key parameters for the analysis are :

| Symbol | Parameter |
|--------|-----------|
| $D$    | Annual Demand    |
| $S$    | Ordering Cost |
| $H$    | Holding Cost  |
| $Q$    | Optimal Order Quantity |
|$L$     | Lead Time |
|$safety$| Safety |
| $d$    | Mean Demand |
| $\sigma$ | Standard Deviation |
| $T$     | Total number of orders |
| $RP$    | Reordering points |
| $N$     | Number of cycles   |
|$AOC$    | Annual Ordering Cost|
|$AHC$    | Annual Holding Cost |
|$ATC$    | Annual Total Cost |


In [2]:
# EOQ Formula
D = 5000
S = 10
H = 0.3 + random.uniform(0,1)
Q = math.sqrt((2*D*S)/H)

# Mean demand rate and standard deviation
d = 100
sigma = 2

# Cost analysis
T = math.ceil(D/Q)
RP = math.ceil(Q/d)
N = int(T*RP)
AOC = (T*S)
AHC = ((Q/2)*H)
ATC = AOC + AHC

# Starting values
L = 2
currentInventory =  500.0
previousInventory =  501.0
safety = Q//4

# Trackers and plotters
tracker = 99999999
currentIteration = 0
inventoryLevel = []
reorderPoints = []
reorderPoint = 0
belowZero = []
isOrder = False
isZero = False
rpX = []
rpY = []
bzX = []
bzY = []

# Statements
display(Markdown(f"Chosen annual demand **(D)** : **{D}**, chosen mean demand **(d)** rate : **{d}**"))
display(Markdown(f"Optimal Order Quantity **(Q)** : **{Q:.2f}**, total number of orders required **(T)** : **{T}**, **(D/Q)**"))
display(Markdown(f"The length of a replenishment cycle **(RP)** is : **{RP}**, **(Q/d)**"))
display(Markdown(f"Number of cycles required to fulfill the annual demand is : **{N}**, **(T x RP)**"))
display(Markdown(f"Fixed Ordering Cost **S** : **{S}**, Varible Holding Cost **H** : **{H:.2f}**, lead time **L** : **{L}**"))
display(Markdown(f"Annual Ordering Cost **(AOC)** : **{AOC:.2f}**"))
display(Markdown(f"Annual Holding Cost **(AHC)** : **{AHC:.2f}**"))
display(Markdown(f"Annual Total Cost **(ATC)** : **{ATC:.2f}**"))

Chosen annual demand **(D)** : **5000**, chosen mean demand **(d)** rate : **100**

Optimal Order Quantity **(Q)** : **376.76**, total number of orders required **(T)** : **14**, **(D/Q)**

The length of a replenishment cycle **(RP)** is : **4**, **(Q/d)**

Number of cycles required to fulfill the annual demand is : **56**, **(T x RP)**

Fixed Ordering Cost **S** : **10**, Varible Holding Cost **H** : **0.70**, lead time **L** : **2**

Annual Ordering Cost **(AOC)** : **140.00**

Annual Holding Cost **(AHC)** : **132.71**

Annual Total Cost **(ATC)** : **272.71**

### Simulation

In [3]:
for cycle in range(N):

    currentIteration = 0
    isOrder = False
    isZero = False
    previousInventory = currentInventory + 1
    
    while (currentInventory < previousInventory):
        randomDemand = d + sigma * np.random.randn(1)

        inventoryLevel.append(currentInventory)
        previousInventory = currentInventory
        currentInventory -= randomDemand.item()
 
        if currentInventory < 0 and not isZero:
            bzX.append(len(inventoryLevel) - 1)
            bzY.append(previousInventory)
            isZero = True
            
        if (currentInventory - (L * d) <= safety) and not isOrder:
            tracker = currentIteration
            reorderPoint = tracker
            rpX.append(len(inventoryLevel))
            rpY.append(currentInventory)
            isOrder = True

        if (currentIteration == tracker + L):
            currentInventory += Q
            isOrder = False
            
        currentIteration += 1
    #break #Use to show reorder point and inventory level before going below zero
    

(array([  8.,  27., 123., 193., 248., 204., 150.,  40.,   5.,   2.]),
 array([308. , 327.8, 347.6, 367.4, 387.2, 407. , 426.8, 446.6, 466.4,
        486.2, 506. ]),
 <BarContainer object of 10 artists>)

In [6]:
d = (np.random.randn(1000)*30+400).astype(int)# is nominally (average) 400 but standard deviation is 30
figX,axX =plt.subplots()
axX.hist(d)

### Plotting

In [4]:
%matplotlib inline
%matplotlib widget
plt.close('all')
num = "EOQ trial"
fig,ax = plt.subplots(num=num, figsize = (16,8))
ax.plot(inventoryLevel, label ="Inventory level")
ax.set(xlabel = "Cycles $(iterations)$", ylabel = "Inventory Level $(units)$")
ax.axhline(Q,color="darkolivegreen", label = "Q")
ax.scatter(rpX,rpY, marker = "^", color = "darkgreen", label = "Reorder points")
# ax.axhline(statistics.mean(inventoryLevel),color = "royalblue", label = "Average inventory level")
ax.scatter(bzX, bzY, marker = "o", color = "orangered", label = "Before below zero")
ax.axhline(safety, color = "red", label = "Safety stock level")
ax.legend(ncol=5);